In [12]:
!pip3 install settings

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement settings (from versions: none)
ERROR: No matching distribution found for settings
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [13]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from textwrap import dedent
from langchain.output_parsers import OutputFixingParser
from langchain.schema import OutputParserException
from langchain_openai import ChatOpenAI
from langfuse.callback import CallbackHandler
from settings import settings

ModuleNotFoundError: No module named 'settings'

In [ ]:
langfuse_handler = CallbackHandler(
    secret_key=settings.LANGFUSE_SECRET_KEY,
    public_key=settings.LANGFUSE_PUBLIC_KEY,
    host=settings.LANGFUSE_HOST, 
)

def parse_response(output, parser, llm=ChatOpenAI(model="gpt-4o", openai_api_key=settings.OPENAI_API_KEY)):
    try:
        parsed = parser.parse(output)
    except OutputParserException:
        try:
            new_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)
            parsed = new_parser.parse(output.content)
            print("Fixed parsing errors.")
        except Exception as e:
            print("Failed to fix parsing errors.")
    return parsed

def llm_invoke(llm, messages):
    return llm.invoke(messages.to_messages(), {"callbacks": [langfuse_handler]})

class ResponseModel(BaseModel):
    soft_skills: List[str] = Field(
        description="List of soft skills keywords found in the job description"
    )
    
    hard_skills: List[str] = Field(
        description="List of hard skills keywords found in the job description"
    )
    
    other_skills: Optional[List[str]] = Field(
        description="List of other keywords, which are not categorized as hard skills or soft skills, but are essential for the job"
    )

def get_overall_keywords(text, llm):
    parser = PydanticOutputParser(pydantic_object=ResponseModel)
    format_instructions = parser.get_format_instructions()
    prompt_template = dedent(
        """
        You are an AI assistant specialized in .....

        Format your response as follows:
        {format_instructions}

        If no keywords fit a particular category, return an empty list for that category. Your goal is to extract accurate and concise keywords that improve ATS matching.

        Text:
        {text}"""
    )

    prompt = ChatPromptTemplate(
        messages=[HumanMessagePromptTemplate.from_template(prompt_template)],
        input_variables=["job_description", "format_instructions"],
    )

    _input = prompt.format_prompt(
        text=text, format_instructions=format_instructions
    )
    # output = llm.invoke(_input.to_messages())
    output = llm_invoke(llm, _input)

    parsed = parse_response(output.content, parser)

    return parsed